<a href="https://colab.research.google.com/github/gusya-soc/notebook_collection/blob/main/VGG16_cifar100_alpha_1_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import cifar100
import numpy as np

(x_train,y_train),(x_test,y_test) = cifar100.load_data()
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
print(x_train.shape,y_train.shape)
print(x_test.dtype)

169009152/169001437 [==============================] - 3s 0us/step
(50000, 32, 32, 3) (50000, 1)
float32


In [ ]:
from keras.utils import np_utils as utl
n_class = 100

y_train = utl.to_categorical(y_train,n_class)
y_test = utl.to_categorical(y_test,n_class)
print(y_test.shape)

(10000, 100)


In [ ]:
from keras.models import Model,load_model,Sequential
from keras.layers import Input, Activation, merge, Dense, Flatten,Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import BatchNormalization, add, GlobalAveragePooling2D,SpatialDropout2D
from keras.callbacks import EarlyStopping,LearningRateScheduler
from keras import regularizers
from keras import optimizers

In [ ]:
class conv_block(Model):
    def __init__(self,filter,is_pool=False,is_bn=True):
        super().__init__()
        self.is_pool = is_pool
        self.filter = filter
        self.is_bn = is_bn
        self.conv = Conv2D(self.filter,kernel_size=3,padding='same',strides=STRIDE,kernel_regularizer=regularizers.l2(weight_decay),kernel_initializer=glorot_normal())
        self.bn   = BatchNormalization(epsilon=1e-3,momentum=0.95,trainable=True)
        self.act  = Activation('relu')
        self.pool = MaxPooling2D(pool_size=(2,2))
        self.drop = SpatialDropout2D(DROP_RATE)

    def call(self,inputs):
        x = self.conv(inputs)
        if self.is_bn:
            x = self.bn(x)
        if self.is_pool:
            x = self.pool(x)
        x = self.act(x)
        x = self.drop(x)
        return x

In [ ]:
class encode_block(conv_block):
    def __init__(self,filter,is_pool=False,is_bn=True):
        super().__init__(filter,is_pool,is_bn)
        self.conv = Conv2D(self.filter,kernel_size=1,strides=1,padding='same',kernel_regularizer=regularizers.l2(weight_decay))


In [ ]:
class net(Model):
    def __init__(self):
        super().__init__()
        # self.inputs = Input(Input_shape)
        self.block_1 = conv_block(64)
        self.block_2 = conv_block(128)
        
        self.block_3 = conv_block(128)
        self.block_4 = conv_block(128,True)
        
        self.block_5 = conv_block(128)
        self.block_6 = conv_block(128)
        self.block_7 = conv_block(256,True)
        self.block_8 = conv_block(256)
        self.block_9 = conv_block(256,True)
        self.block_10 = conv_block(512)
        self.block_11 = encode_block(2048,is_bn=False)
        self.block_12 = encode_block(256,is_pool=True,is_bn=False)
        self.block_13 = conv_block(256,is_pool=True,is_bn=False)
        self.fc = Flatten()
        self.dc = Dense(100,activation='softmax')
        
    def call(self,inputs):
        # x = self.inputs(inputs)
        x = self.block_1(inputs)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.block_4(x)
        x = self.block_5(x)
        x = self.block_6(x)
        x = self.block_7(x)
        x = self.block_8(x)
        x = self.block_9(x)
        x = self.block_10(x)
        x = self.block_11(x)
        x = self.block_12(x)
        x = self.block_13(x)
        # x = self.conv_13(x)
        # x = self.pool_13(x)
        # x = self.drop_13(x)
        x = self.fc(x)
        x = self.dc(x)
        return x

In [ ]:
STRIDE = (1,1)
KSIZE = (3,3) 
weight_decay = 5e-3
DROP_RATE = 0.2
Input_shape = (32,32,3)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )
datagen.fit(x_train)

In [ ]:
learning_rate = 1e-3
lr_decay = 2e-5
lr_drop = 20
sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
lr_base = 0.1
opt_adm = keras.optimizers.Adadelta(lr=0.1)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
def lr_scheduler(epoch):
    return learning_rate * (0.5 ** (epoch // lr_drop))
reduce_lr = LearningRateScheduler(lr_scheduler)
callbacks=[reduce_lr]

In [ ]:
model = net()
model.build(input_shape=(None,32,32,3))
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "net_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_block_26 (conv_block)   multiple                  2048      
_________________________________________________________________
conv_block_27 (conv_block)   multiple                  74368     
_________________________________________________________________
conv_block_28 (conv_block)   multiple                  148096    
_________________________________________________________________
conv_block_29 (conv_block)   multiple                  148096    
_________________________________________________________________
conv_block_30 (conv_block)   multiple                  148096    
_________________________________________________________________
conv_block_31 (conv_block)   multiple                  148096    
_________________________________________________________________
conv_block_32 (conv_block)   multiple                  296192

In [ ]:
model.compiley

In [ ]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
y_train.shape

(50000, 100)

In [ ]:
model.fit(x_train, y_train, batch_size=200, epochs=50, verbose=1, validation_split=0.1,shuffle=True,callbacks=callbacks)

Epoch 1/50
225/225 [==============================] - 30s 90ms/step - loss: 6.2553 - accuracy: 0.0107 - val_loss: 6.1771 - val_accuracy: 0.0094
Epoch 2/50
225/225 [==============================] - 20s 89ms/step - loss: 6.1830 - accuracy: 0.0117 - val_loss: 6.1705 - val_accuracy: 0.0096
Epoch 3/50
225/225 [==============================] - 20s 90ms/step - loss: 6.1496 - accuracy: 0.0142 - val_loss: 6.1595 - val_accuracy: 0.0104
Epoch 4/50
225/225 [==============================] - 20s 90ms/step - loss: 6.1048 - accuracy: 0.0170 - val_loss: 6.1506 - val_accuracy: 0.0114
Epoch 5/50
225/225 [==============================] - 20s 89ms/step - loss: 6.0832 - accuracy: 0.0180 - val_loss: 6.1427 - val_accuracy: 0.0132
Epoch 6/50
225/225 [==============================] - 20s 89ms/step - loss: 6.0682 - accuracy: 0.0174 - val_loss: 6.1411 - val_accuracy: 0.0126
Epoch 7/50
225/225 [==============================] - 20s 90ms/step - loss: 6.0455 - accuracy: 0.0202 - val_loss: 6.1402 - val_accuracy:

In [ ]:
model.fit(x_train, y_train, batch_size=200, epochs=50, verbose=1, validation_split=0.1,shuffle=True,callbacks=callbacks)

Epoch 1/50
225/225 [==============================] - 20s 90ms/step - loss: 5.6607 - accuracy: 0.0301 - val_loss: 7.1702 - val_accuracy: 0.0118
Epoch 2/50
225/225 [==============================] - 20s 89ms/step - loss: 5.6512 - accuracy: 0.0297 - val_loss: 7.1143 - val_accuracy: 0.0082
Epoch 3/50
225/225 [==============================] - 20s 90ms/step - loss: 5.6427 - accuracy: 0.0290 - val_loss: 7.0884 - val_accuracy: 0.0088
Epoch 4/50
225/225 [==============================] - 20s 90ms/step - loss: 5.6307 - accuracy: 0.0305 - val_loss: 7.0639 - val_accuracy: 0.0086
Epoch 5/50
225/225 [==============================] - 20s 90ms/step - loss: 5.6233 - accuracy: 0.0309 - val_loss: 6.8994 - val_accuracy: 0.0090
Epoch 6/50
225/225 [==============================] - 20s 90ms/step - loss: 5.6182 - accuracy: 0.0301 - val_loss: 6.9043 - val_accuracy: 0.0090
Epoch 7/50
225/225 [==============================] - 20s 90ms/step - loss: 5.6069 - accuracy: 0.0303 - val_loss: 6.8233 - val_accuracy:

In [ ]:
model.fit(x_train, y_train, batch_size=200, epochs=50, verbose=1, validation_split=0.1,shuffle=True,callbacks=callbacks)

Epoch 1/50
225/225 [==============================] - 20s 88ms/step - loss: 5.3322 - accuracy: 0.0506 - val_loss: 5.6281 - val_accuracy: 0.0378
Epoch 2/50
225/225 [==============================] - 20s 88ms/step - loss: 5.3197 - accuracy: 0.0529 - val_loss: 5.7325 - val_accuracy: 0.0368
Epoch 3/50
225/225 [==============================] - 20s 88ms/step - loss: 5.3094 - accuracy: 0.0514 - val_loss: 5.6080 - val_accuracy: 0.0394
Epoch 4/50
225/225 [==============================] - 20s 88ms/step - loss: 5.3055 - accuracy: 0.0528 - val_loss: 5.6044 - val_accuracy: 0.0412
Epoch 5/50
225/225 [==============================] - 20s 88ms/step - loss: 5.2930 - accuracy: 0.0525 - val_loss: 5.5802 - val_accuracy: 0.0436
Epoch 6/50
225/225 [==============================] - 20s 88ms/step - loss: 5.2873 - accuracy: 0.0552 - val_loss: 5.5320 - val_accuracy: 0.0438
Epoch 7/50
225/225 [==============================] - 20s 88ms/step - loss: 5.2755 - accuracy: 0.0560 - val_loss: 5.5898 - val_accuracy:

In [ ]:
model.fit(x_train, y_train, batch_size=200, epochs=50, verbose=1, validation_split=0.1,shuffle=True,callbacks=callbacks)

Epoch 1/50
225/225 [==============================] - 20s 88ms/step - loss: 5.0650 - accuracy: 0.0748 - val_loss: 5.2489 - val_accuracy: 0.0652
Epoch 2/50
225/225 [==============================] - 20s 88ms/step - loss: 5.0544 - accuracy: 0.0746 - val_loss: 5.2173 - val_accuracy: 0.0678
Epoch 3/50
225/225 [==============================] - 20s 88ms/step - loss: 5.0445 - accuracy: 0.0761 - val_loss: 5.2086 - val_accuracy: 0.0650
Epoch 4/50
225/225 [==============================] - 20s 88ms/step - loss: 5.0395 - accuracy: 0.0758 - val_loss: 5.1575 - val_accuracy: 0.0704
Epoch 5/50
225/225 [==============================] - 20s 88ms/step - loss: 5.0356 - accuracy: 0.0765 - val_loss: 5.2001 - val_accuracy: 0.0682
Epoch 6/50
225/225 [==============================] - 20s 88ms/step - loss: 5.0321 - accuracy: 0.0763 - val_loss: 5.1460 - val_accuracy: 0.0716
Epoch 7/50
225/225 [==============================] - 20s 88ms/step - loss: 5.0188 - accuracy: 0.0782 - val_loss: 5.1759 - val_accuracy:

In [ ]:
model.fit(x_train, y_train, batch_size=250, epochs=50, verbose=1, validation_split=0.1,shuffle=True,callbacks=callbacks)

Epoch 1/50
180/180 [==============================] - 19s 103ms/step - loss: 4.2026 - accuracy: 0.1778 - val_loss: 4.0406 - val_accuracy: 0.2156
Epoch 2/50
180/180 [==============================] - 19s 103ms/step - loss: 4.2017 - accuracy: 0.1756 - val_loss: 4.0665 - val_accuracy: 0.2116
Epoch 3/50
180/180 [==============================] - 19s 104ms/step - loss: 4.1920 - accuracy: 0.1802 - val_loss: 4.0562 - val_accuracy: 0.2136
Epoch 4/50
180/180 [==============================] - 19s 104ms/step - loss: 4.1890 - accuracy: 0.1788 - val_loss: 4.0404 - val_accuracy: 0.2140
Epoch 5/50
180/180 [==============================] - 19s 105ms/step - loss: 4.1843 - accuracy: 0.1787 - val_loss: 4.0185 - val_accuracy: 0.2224
Epoch 6/50
180/180 [==============================] - 19s 105ms/step - loss: 4.1768 - accuracy: 0.1825 - val_loss: 4.0153 - val_accuracy: 0.2208
Epoch 7/50
180/180 [==============================] - 19s 105ms/step - loss: 4.1760 - accuracy: 0.1804 - val_loss: 4.0162 - val_ac

In [ ]:
for i in range(5):
    model.fit(x_train, y_train, batch_size=250, epochs=50, verbose=1, validation_split=0.1,shuffle=True,callbacks=callbacks)
    print('='*40)
    print('\n')

Epoch 1/50
180/180 [==============================] - 19s 106ms/step - loss: 4.0358 - accuracy: 0.2013 - val_loss: 3.8589 - val_accuracy: 0.2442
Epoch 2/50
180/180 [==============================] - 19s 106ms/step - loss: 4.0346 - accuracy: 0.2013 - val_loss: 3.8472 - val_accuracy: 0.2410
Epoch 3/50
180/180 [==============================] - 19s 106ms/step - loss: 4.0318 - accuracy: 0.2012 - val_loss: 3.8566 - val_accuracy: 0.2380
Epoch 4/50
180/180 [==============================] - 19s 106ms/step - loss: 4.0280 - accuracy: 0.2006 - val_loss: 3.8639 - val_accuracy: 0.2422
Epoch 5/50
180/180 [==============================] - 19s 107ms/step - loss: 4.0226 - accuracy: 0.2023 - val_loss: 3.8296 - val_accuracy: 0.2470
Epoch 6/50
180/180 [==============================] - 19s 106ms/step - loss: 4.0124 - accuracy: 0.2041 - val_loss: 3.8403 - val_accuracy: 0.2432
Epoch 7/50
180/180 [==============================] - 19s 106ms/step - loss: 4.0081 - accuracy: 0.2011 - val_loss: 3.8280 - val_ac

In [ ]:
for i in range(10):
    model.fit(x_train, y_train, batch_size=250, epochs=50, verbose=1, validation_split=0.1,shuffle=True,callbacks=callbacks)
    print('='*40)
    print('\n')